In [25]:
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.datasets import load_wine
from AdjustedRandomForest import train

In [26]:
# MISLABELLING
scores_my = []
mismatched_my = []
scores_std = []
MISLABELLING = 0.6

for _ in range(100):
    wine = load_wine()
    X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target)

    y_mislabelled = np.copy(y_train)

    for i in range(int(len(y_mislabelled) * MISLABELLING)):
        y_mislabelled[i] = (y_mislabelled[0] + random.randint(1,2)) % len(np.unique(wine.target))

    rf, corrected_y = train(RandomForestClassifier, X_train, y_mislabelled, np.unique(wine.target), n_estimators=5)
    
    y_test_pred = rf.predict(X_test)

    #mismatch_indices = [index for index, (i, j) in enumerate(zip(corrected_y, y_train)) if i != j]

    mismatched_my.append(sum(i!=j for i, j in zip(corrected_y, y_train)))
    scores_my.append(metrics.accuracy_score(y_test, y_test_pred))

    rf = RandomForestClassifier(n_estimators=100, criterion='entropy')
    rf.fit(X_train, y_mislabelled)
    y_test_pred = rf.predict(X_test)
    scores_std.append(metrics.accuracy_score(y_test, y_test_pred))

mismatched_init = int(len(X_train) * MISLABELLING)

print(f"\nTest Average Accuracy: {(np.mean(scores_my)*100).round(2)}%")
print(f"Labels Corrected: {((1 - np.mean(mismatched_my)/mismatched_init) * 100).round(2)}%")
print(f"Control Average Accuracy: {(np.mean(scores_std)*100).round(2)}%")

# Is accuracy about 50% of data at no mislabelling
# Does evaluating off n trees make any difference?


Test Average Accuracy: 79.53%
Labels Corrected: 80.47%
Control Average Accuracy: 72.38%
